<a href="https://colab.research.google.com/github/sahil97/30MinTask/blob/master/brinks_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import pandas as pd
import numpy as np
import tensorflow as tf
import nltk, re, time
from nltk.corpus import stopwords
from string import punctuation
from collections import defaultdict
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from collections import namedtuple

In [36]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [0]:

imdb_train = pd.read_csv("gdrive/My Drive/ML Data/labeledTrainData.tsv", delimiter="\t")
amazon_train = pd.read_csv('gdrive/My Drive/ML Data/amazon_cells_labelled.txt', delimiter="\t",header=-1)
yelp_train = pd.read_csv('gdrive/My Drive/ML Data/yelp_labelled.txt',delimiter="\t",header=-1)


test = pd.read_csv("gdrive/My Drive/ML Data/testData.tsv", delimiter="\t")

In [49]:
imdb_train.drop(['id'],axis=1,inplace=True)
imdb_train.head()

,sentiment,review
0,1,With all this stuff going down at the moment w...
1,1,"\The Classic War of the Worlds\"" by Timothy Hi..."
2,0,The film starts with a manager (Nicholas Bell)...
3,0,It must be assumed that those who praised this...
4,1,Superbly trashy and wondrously unpretentious 8...


In [0]:
# amazon_train.head()
# amazon_train['sentiment'] = amazon_train[1]
# amazon_train['review']=amazon_train[0]
# amazon_train.drop([0,1],axis=1,inplace=True)

In [0]:
#amazon_train.head()

In [0]:
# yelp_train.head()
# yelp_train['sentiment'] = yelp_train[1]
# yelp_train['review']=yelp_train[0]
# yelp_train.drop([0,1],axis=1,inplace=True)

In [0]:
# yelp_train.head()

In [0]:
# test.head()

In [0]:
# # print(amazon_train.shape)
# print(yelp_train.shape)
# print(imdb_train.shape)

In [0]:
train = pd.read_csv("gdrive/My Drive/ML Data/train_data.csv", delimiter="\t")

In [57]:
print(train.shape)
print(test.shape)

(21000, 3)
(25000, 2)


In [58]:

for i in range(3):
    print(train.review[i])
    print()

With all this stuff going down at the moment with MJ i've started listening to his music, watching the odd documentary here and there, watched The Wiz and watched Moonwalker again. Maybe i just want to get a certain insight into this guy who i thought was really cool in the eighties just to maybe make up my mind whether he is guilty or innocent. Moonwalker is part biography, part feature film which i remember going to see at the cinema when it was originally released. Some of it has subtle messages about MJ's feeling towards the press and also the obvious message of drugs are bad m'kay.<br /><br />Visually impressive but of course this is all about Michael Jackson so unless you remotely like MJ in anyway then you are going to hate this and find it boring. Some may call MJ an egotist for consenting to the making of this movie BUT MJ and most of his fans would say that he made it for the fans which if true is really nice of him.<br /><br />The actual feature film bit when it finally star

In [59]:
# Check for any null values
print(train.isnull().sum())
print(test.isnull().sum())

Unnamed: 0    0
sentiment     0
review        0
dtype: int64
id        0
review    0
dtype: int64


In [0]:
def clean_text(text, remove_stopwords=True):
    '''Clean the text, with the option to remove stopwords'''
    
    # Convert words to lower case and split them
    text = text.lower().split()

    # Optionally, remove stop words
    if remove_stopwords:
        stops = set(stopwords.words("english"))
        text = [w for w in text if not w in stops]
    
    text = " ".join(text)

    # Clean the text
    text = re.sub(r"<br />", " ", text)
    text = re.sub(r"[^a-z]", " ", text)
    text = re.sub(r"   ", " ", text) # Remove any extra spaces
    text = re.sub(r"  ", " ", text)
    
    # Remove punctuation from text
    text = ''.join([c for c in text if c not in punctuation])
    
    # Return a list of words
    return(text)

In [61]:
import nltk
nltk.download('stopwords')
train_clean = []
for review in train.review:
    train_clean.append(clean_text(review))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [0]:
test_clean = []
for review in test.review:
    test_clean.append(clean_text(review))

In [63]:
# Inspect the cleaned reviews
for i in range(3):
    print(train_clean[i])
    print()

stuff going moment mj i ve started listening music watching odd documentary there watched wiz watched moonwalker again maybe want get certain insight guy thought really cool eighties maybe make mind whether guilty innocent moonwalker part biography part feature film remember going see cinema originally released subtle messages mj s feeling towards press also obvious message drugs bad m kay visually impressive course michael jackson unless remotely like mj anyway going hate find boring may call mj egotist consenting making movie mj fans would say made fans true really nice him the actual feature film bit finally starts minutes excluding smooth criminal sequence joe pesci convincing psychopathic powerful drug lord wants mj dead bad beyond me mj overheard plans nah joe pesci s character ranted wanted people know supplying drugs etc dunno maybe hates mj s music lots cool things like mj turning car robot whole speed demon sequence also director must patience saint came filming kiddy bad seq

In [64]:
len(train_clean)

21000

In [65]:
# Tokenize the reviews
all_reviews = train_clean + test_clean
tokenizer = Tokenizer()
tokenizer.fit_on_texts(all_reviews)
print("Fitting is complete.")

train_seq = tokenizer.texts_to_sequences(train_clean)
print("train_seq is complete.")

test_seq = tokenizer.texts_to_sequences(test_clean)
print("test_seq is complete")

Fitting is complete.
train_seq is complete.
test_seq is complete


In [66]:
# Find the number of unique tokens
word_index = tokenizer.word_index
print("Words in index: %d" % len(word_index))

Words in index: 93246


In [67]:
# Inspect the reviews after they have been tokenized
for i in range(3):
    print(train_seq[i])
    print()

[388, 66, 530, 10358, 6, 59, 502, 2614, 115, 83, 966, 661, 49, 243, 19748, 243, 22043, 189, 177, 88, 17, 716, 2891, 122, 108, 12, 386, 4298, 177, 24, 253, 710, 2383, 1448, 22043, 97, 5178, 97, 789, 3, 308, 66, 15, 409, 1958, 649, 1333, 4233, 10358, 1, 483, 929, 3435, 20, 584, 758, 1549, 22, 50, 6396, 2478, 1165, 178, 481, 1758, 760, 2778, 5, 10358, 463, 66, 658, 72, 309, 118, 495, 10358, 40595, 36243, 160, 2, 10358, 423, 11, 56, 25, 423, 228, 12, 130, 201, 10, 808, 789, 3, 112, 328, 527, 128, 16923, 3867, 2034, 736, 877, 12358, 1218, 13395, 989, 1118, 1911, 469, 10358, 300, 22, 639, 110, 10358, 12812, 2296, 16438, 877, 12358, 1, 44, 36244, 315, 21, 37, 16924, 1549, 402, 8908, 177, 4529, 10358, 1, 115, 541, 386, 96, 5, 10358, 1719, 404, 2647, 138, 2212, 2703, 736, 20, 92, 117, 4632, 5790, 197, 1497, 28498, 22, 736, 425, 1040, 658, 620, 4, 467, 184, 572, 138, 706, 3775, 1292, 855, 63, 1199, 267, 2, 21, 5, 10358, 4, 590, 70, 464, 28, 21, 248, 632, 153, 165, 99, 8019, 758, 3880, 10358, 1, 

In [0]:
# Find the length of reviews
lengths = []
for review in train_seq:
    lengths.append(len(review))

for review in test_seq:
    lengths.append(len(review))

# Create a dataframe so that the values can be inspected
lengths = pd.DataFrame(lengths, columns=['counts'])

In [69]:
lengths.counts.describe()

count    46000.000000
mean       112.870109
std         93.428205
min          1.000000
25%         59.000000
50%         85.000000
75%        139.000000
max       1488.000000
Name: counts, dtype: float64

In [70]:
print(np.percentile(lengths.counts, 80))
print(np.percentile(lengths.counts, 85))
print(np.percentile(lengths.counts, 90))
print(np.percentile(lengths.counts, 95))

160.0
187.0
228.0
305.0


In [71]:
# Pad and truncate the questions so that they all have the same length.
max_review_length = 200

train_pad = pad_sequences(train_seq, maxlen = max_review_length)
print("train_pad is complete.")

test_pad = pad_sequences(test_seq, maxlen = max_review_length)
print("test_pad is complete.")

train_pad is complete.
test_pad is complete.


In [72]:
# Inspect the reviews after padding has been completed. 
for i in range(3):
    print(train_pad[i,:100])
    print()

[   97  5178    97   789     3   308    66    15   409  1958   649  1333
  4233 10358     1   483   929  3435    20   584   758  1549    22    50
  6396  2478  1165   178   481  1758   760  2778     5 10358   463    66
   658    72   309   118   495 10358 40595 36243   160     2 10358   423
    11    56    25   423   228    12   130   201    10   808   789     3
   112   328   527   128 16923  3867  2034   736   877 12358  1218 13395
   989  1118  1911   469 10358   300    22   639   110 10358 12812  2296
 16438   877 12358     1    44 36244   315    21    37 16924  1549   402
  8908   177  4529 10358]

[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]

[ 7057  4986   363  7854  8256  4088  2179  7694  2194 13090   564  2348
  1512  3405  2695   605  1675  1396  1727 30536  1736  1528  2348  1562
  8617 36246  4751  2038 58152   565  

In [0]:
# Creating the training and validation sets
x_train, x_valid, y_train, y_valid = train_test_split(train_pad, train.sentiment, test_size = 0.15, random_state = 2)
x_test = test_pad

In [74]:
# Inspect the shape of the data
print(x_train.shape)
print(x_valid.shape)
print(x_test.shape)

(17850, 200)
(3150, 200)
(25000, 200)


In [0]:
def get_batches(x, y, batch_size):
    '''Create the batches for the training and validation data'''
    n_batches = len(x)//batch_size
    x, y = x[:n_batches*batch_size], y[:n_batches*batch_size]
    for ii in range(0, len(x), batch_size):
        yield x[ii:ii+batch_size], y[ii:ii+batch_size]

In [0]:
def get_test_batches(x, batch_size):
    '''Create the batches for the testing data'''
    n_batches = len(x)//batch_size
    x = x[:n_batches*batch_size]
    for ii in range(0, len(x), batch_size):
        yield x[ii:ii+batch_size]

In [0]:
def lstm_cell(lstm_size, keep_prob):
    lstm = tf.contrib.rnn.BasicLSTMCell(lstm_size)
    drop = tf.contrib.rnn.DropoutWrapper(lstm, output_keep_prob=keep_prob)
    return drop

In [0]:
def build_rnn(n_words, embed_size, batch_size, lstm_size, num_layers, 
              dropout, learning_rate, multiple_fc, fc_units):
    '''Build the Recurrent Neural Network'''

    tf.reset_default_graph()

    # Declare placeholders we'll feed into the graph
    with tf.name_scope('inputs'):
        inputs = tf.placeholder(tf.int32, [None, None], name='inputs')

    with tf.name_scope('labels'):
        labels = tf.placeholder(tf.int32, [None, None], name='labels')

    keep_prob = tf.placeholder(tf.float32, name='keep_prob')

    # Create the embeddings
    with tf.name_scope("embeddings"):
        embedding = tf.Variable(tf.random_uniform((n_words, embed_size), -1, 1))
        embed = tf.nn.embedding_lookup(embedding, inputs)

    # Build the RNN layers
    with tf.name_scope("RNN_layers"):
        cell = tf.contrib.rnn.MultiRNNCell([lstm_cell(lstm_size, keep_prob) for _ in range(num_layers)])
    
    # Set the initial state
    with tf.name_scope("RNN_init_state"):
        initial_state = cell.zero_state(batch_size, tf.float32)

    # Run the data through the RNN layers
    with tf.name_scope("RNN_forward"):
        outputs, final_state = tf.nn.dynamic_rnn(cell, embed,
                                                 initial_state=initial_state)    
    
    # Create the fully connected layers
    with tf.name_scope("fully_connected"):
        
        # Initialize the weights and biases
        weights = tf.truncated_normal_initializer(stddev=0.1)
        biases = tf.zeros_initializer()
        
        dense = tf.contrib.layers.fully_connected(outputs[:, -1],
                                                  num_outputs = fc_units,
                                                  activation_fn = tf.sigmoid,
                                                  weights_initializer = weights,
                                                  biases_initializer = biases)
        dense = tf.contrib.layers.dropout(dense, keep_prob)
        
        # Depending on the iteration, use a second fully connected layer
        if multiple_fc == True:
            dense = tf.contrib.layers.fully_connected(dense,
                                                      num_outputs = fc_units,
                                                      activation_fn = tf.sigmoid,
                                                      weights_initializer = weights,
                                                      biases_initializer = biases)
            dense = tf.contrib.layers.dropout(dense, keep_prob)
    
    # Make the predictions
    with tf.name_scope('predictions'):
        predictions = tf.contrib.layers.fully_connected(dense, 
                                                        num_outputs = 1, 
                                                        activation_fn=tf.sigmoid,
                                                        weights_initializer = weights,
                                                        biases_initializer = biases)
        tf.summary.histogram('predictions', predictions)
    
    # Calculate the cost
    with tf.name_scope('cost'):
        cost = tf.losses.mean_squared_error(labels, predictions)
        tf.summary.scalar('cost', cost)
    
    # Train the model
    with tf.name_scope('train'):    
        optimizer = tf.train.AdamOptimizer(learning_rate).minimize(cost)

    # Determine the accuracy
    with tf.name_scope("accuracy"):
        correct_pred = tf.equal(tf.cast(tf.round(predictions), tf.int32), labels)
        accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))
        tf.summary.scalar('accuracy', accuracy)
    
    # Merge all of the summaries
    merged = tf.summary.merge_all()    

    # Export the nodes 
    export_nodes = ['inputs', 'labels', 'keep_prob', 'initial_state', 'final_state','accuracy',
                    'predictions', 'cost', 'optimizer', 'merged']
    Graph = namedtuple('Graph', export_nodes)
    local_dict = locals()
    graph = Graph(*[local_dict[each] for each in export_nodes])
    
    return graph

In [79]:
!mkdir 'gdrive/My Drive/Model_checkpoints/trial2'

mkdir: cannot create directory ‘gdrive/My Drive/Model_checkpoints/trial2’: File exists


In [0]:
def train(model, epochs, log_string):
    '''Train the RNN'''

    saver = tf.train.Saver()
    
    with tf.Session() as sess:
        sess.run(tf.global_variables_initializer())

        # Used to determine when to stop the training early
        valid_loss_summary = []
        
        # Keep track of which batch iteration is being trained
        iteration = 0

        print()
        print("Training Model: {}".format(log_string))

        train_writer = tf.summary.FileWriter('./logs/3/train/{}'.format(log_string), sess.graph)
        valid_writer = tf.summary.FileWriter('./logs/3/valid/{}'.format(log_string))

        for e in range(epochs):
            state = sess.run(model.initial_state)
            
            # Record progress with each epoch
            train_loss = []
            train_acc = []
            val_acc = []
            val_loss = []

            with tqdm(total=len(x_train)) as pbar:
                for _, (x, y) in enumerate(get_batches(x_train, y_train, batch_size), 1):
                    feed = {model.inputs: x,
                            model.labels: y[:, None],
                            model.keep_prob: dropout,
                            model.initial_state: state}
                    summary, loss, acc, state, _ = sess.run([model.merged, 
                                                             model.cost, 
                                                             model.accuracy, 
                                                             model.final_state, 
                                                             model.optimizer], 
                                                            feed_dict=feed)                
                    
                    # Record the loss and accuracy of each training batch
                    train_loss.append(loss)
                    train_acc.append(acc)
                    
                    # Record the progress of training
                    train_writer.add_summary(summary, iteration)
                    
                    iteration += 1
                    pbar.update(batch_size)
            
            # Average the training loss and accuracy of each epoch
            avg_train_loss = np.mean(train_loss)
            avg_train_acc = np.mean(train_acc) 

            val_state = sess.run(model.initial_state)
            with tqdm(total=len(x_valid)) as pbar:
                for x, y in get_batches(x_valid, y_valid, batch_size):
                    feed = {model.inputs: x,
                            model.labels: y[:, None],
                            model.keep_prob: 1,
                            model.initial_state: val_state}
                    summary, batch_loss, batch_acc, val_state = sess.run([model.merged, 
                                                                          model.cost, 
                                                                          model.accuracy, 
                                                                          model.final_state], 
                                                                         feed_dict=feed)
                    
                    # Record the validation loss and accuracy of each epoch
                    val_loss.append(batch_loss)
                    val_acc.append(batch_acc)
                    pbar.update(batch_size)
            
            # Average the validation loss and accuracy of each epoch
            avg_valid_loss = np.mean(val_loss)    
            avg_valid_acc = np.mean(val_acc)
            valid_loss_summary.append(avg_valid_loss)
            
            # Record the validation data's progress
            valid_writer.add_summary(summary, iteration)

            # Print the progress of each epoch
            print("Epoch: {}/{}".format(e, epochs),
                  "Train Loss: {:.3f}".format(avg_train_loss),
                  "Train Acc: {:.3f}".format(avg_train_acc),
                  "Valid Loss: {:.3f}".format(avg_valid_loss),
                  "Valid Acc: {:.3f}".format(avg_valid_acc))

            # Stop training if the validation loss does not decrease after 3 epochs
            if avg_valid_loss > min(valid_loss_summary):
                print("No Improvement.")
                stop_early += 1
                if stop_early == 3:
                    break   
            
            # Reset stop_early if the validation loss finds a new low
            # Save a checkpoint of the model
            else:
                print("New Record!")
                stop_early = 0
                checkpoint = "gdrive/My Drive/Model_checkpoints/trial2/sentiment_{}.ckpt".format(log_string)
                saver.save(sess, checkpoint)

In [0]:
# The default parameters of the model
n_words = len(word_index)
embed_size = 300
batch_size = 2
lstm_size = 128
num_layers = 2
dropout = 0.5
learning_rate = 0.01
epochs = 100
multiple_fc = False
fc_units = 256

In [113]:
# Train the model with the desired tuning parameters
for lstm_size in [64,128]:
    for multiple_fc in [True, False]:
        for fc_units in [128, 256]:
            log_string = 'ru={},fcl={},fcu={}'.format(lstm_size,
                                                      multiple_fc,
                                                      fc_units)
            model = build_rnn(n_words = n_words, 
                              embed_size = embed_size,
                              batch_size = batch_size,
                              lstm_size = lstm_size,
                              num_layers = num_layers,
                              dropout = dropout,
                              learning_rate = learning_rate,
                              multiple_fc = multiple_fc,
                              fc_units = fc_units)            
            train(model, epochs, log_string)


Training Model: ru=64,fcl=True,fcu=128


 95%|█████████▌| 3000/3150 [00:02<00:00, 1265.72it/s]


Epoch: 0/100 Train Loss: 0.223 Train Acc: 0.664 Valid Loss: 0.171 Valid Acc: 0.781
New Record!


 48%|████▊     | 8500/17850 [00:14<00:15, 599.59it/s]


KeyboardInterrupt: ignored

In [143]:
brinks_pad

array([[    0,     0,     0, ..., 21100,   683,  9610],
       [    0,     0,     0, ...,  2732,   238, 51085],
       [    0,     0,     0, ...,  7900,  9191,  3656],
       ...,
       [    0,     0,     0, ...,  5191,  4462, 10162],
       [    0,     0,     0, ...,     0,    29,   212],
       [    0,     0,     0, ...,   344,    14,  3439]], dtype=int32)

In [144]:
test_pad

array([[    0,     0,     0, ...,  4291,  3967,   930],
       [    0,     0,     0, ...,     9,  3498,  1555],
       [    0,     0,     0, ..., 17619,     1,   380],
       ...,
       [    0,     0,     0, ...,   228,    18,   391],
       [    0,     0,     0, ...,   200,    29, 10331],
       [    0,     0,     0, ...,   132,  1830,     4]], dtype=int32)

In [0]:
def make_predictions(lstm_size, multiple_fc, fc_units, checkpoint):
    '''Predict the sentiment of the testing data'''
    
    # Record all of the predictions
    all_preds = []

    model = build_rnn(n_words = n_words, 
                      embed_size = embed_size,
                      batch_size = batch_size,
                      lstm_size = lstm_size,
                      num_layers = num_layers,
                      dropout = dropout,
                      learning_rate = learning_rate,
                      multiple_fc = multiple_fc,
                      fc_units = fc_units) 
    
    with tf.Session() as sess:
        saver = tf.train.Saver()
        # Load the model
        saver.restore(sess, checkpoint)
        test_state = sess.run(model.initial_state)
        for _, x in enumerate(get_test_batches(test_pad, batch_size), 1):
            feed = {model.inputs: x,
                    model.keep_prob: 1,
                    model.initial_state: test_state}
            predictions = sess.run(model.predictions, feed_dict=feed)
            for pred in predictions:
                all_preds.append(float(pred))
                
    return all_preds

In [0]:
checkpoint1 = "gdrive/My Drive/Model_checkpoints/trial2/sentiment_ru=128,fcl=False,fcu=256.ckpt"
checkpoint2 = "gdrive/My Drive/Model_checkpoints/trial2/sentiment_ru=128,fcl=False,fcu=128.ckpt"
checkpoint3 = "gdrive/My Drive/Model_checkpoints/trial2/sentiment_ru=64,fcl=True,fcu=256.ckpt"

In [150]:
# Make predictions using the best 3 models
predictions1 = make_predictions(128, False, 256, checkpoint1)
predictions2 = make_predictions(128, False, 128, checkpoint2)
predictions3 = make_predictions(64, True, 256, checkpoint3)

INFO:tensorflow:Restoring parameters from gdrive/My Drive/Model_checkpoints/trial2/sentiment_ru=128,fcl=False,fcu=256.ckpt
INFO:tensorflow:Restoring parameters from gdrive/My Drive/Model_checkpoints/trial2/sentiment_ru=128,fcl=False,fcu=128.ckpt


KeyboardInterrupt: ignored

In [151]:
predictions1

[0.9769830703735352,
 0.011080026626586914,
 0.486637145280838,
 0.5805355906486511,
 0.7314826250076294,
 0.1490570306777954,
 0.21622255444526672,
 0.11603870987892151,
 0.009724825620651245,
 0.03026154637336731,
 0.017169713973999023,
 0.4152761697769165,
 0.17688080668449402,
 0.6220370531082153,
 0.23107418417930603,
 0.8626309633255005,
 0.10771194100379944,
 0.07638686895370483,
 0.9191462993621826,
 0.010332435369491577,
 0.010252624750137329,
 0.6721173524856567,
 0.9686622023582458,
 0.012571275234222412,
 0.6145703196525574,
 0.014154046773910522,
 0.009472072124481201,
 0.8827129602432251,
 0.013550102710723877,
 0.014898091554641724,
 0.9727814793586731,
 0.9761784076690674,
 0.955047607421875,
 0.013719797134399414,
 0.6349088549613953,
 0.976715624332428,
 0.022113382816314697,
 0.01229676604270935,
 0.6900664567947388,
 0.03481161594390869,
 0.07908719778060913,
 0.009900718927383423,
 0.2294045090675354,
 0.9742465019226074,
 0.9760715365409851,
 0.9762983322143555,
 

In [0]:
# # 
predictions_combined = (pd.DataFrame(predictions1) + pd.DataFrame(predictions2) + pd.DataFrame(predictions3))/3

In [143]:
# predictions_combined.head()

,0
0,0.824558
1,0.176947
2,0.576638
3,0.768944
4,0.819436


In [144]:
# test.head()

,id,review
0,12311_10,Naturally in a film who's main themes are of m...
1,8348_2,This movie is a disaster within a disaster fil...
2,5828_4,"All in all, this is a movie for kids. We saw i..."
3,7186_2,Afraid of the Dark left me with the impression...
4,12128_7,A very accurate depiction of small time mob li...


In [0]:
brinks_test = pd.read_excel("gdrive/My Drive/ML Data/brinks_sample (1).xls", delimiter="\t")

In [159]:
len(brinks_test)

1334

In [0]:
def clean_test(text, remove_stopwords=True):
    '''Clean the text, with the option to remove stopwords'''
    
    # Convert words to lower case and split them
    text = str(text).lower().split()

    # Optionally, remove stop words
    if remove_stopwords:
        stops = set(stopwords.words("english"))
        text = [w for w in text if not w in stops]
    
    text = " ".join(text)

    # Clean the text
    text = re.sub(r"<br />", " ", text)
    text = re.sub(r"[^a-z]", " ", text)
    text = re.sub(r"   ", " ", text) # Remove any extra spaces
    text = re.sub(r"  ", " ", text)
    
    # Remove punctuation from text
    text = ''.join([c for c in text if c not in punctuation])
    
    # Return a list of words
    return(text)

In [0]:
brinks_clean = []
for review in brinks_test.Review:
    brinks_clean.append(clean_test(review))

In [162]:
brinks_test_seq = tokenizer.texts_to_sequences(brinks_clean)
print("test_seq is complete")

test_seq is complete


In [163]:
brinks_test_seq[1]

[617, 2501, 11990, 3656]

In [164]:
brinks_clean[1]

'talk customers updating equipment '

In [165]:
brinks_pad = pad_sequences(brinks_test_seq, maxlen = 200)
print("test_pad is complete.")

test_pad is complete.


In [166]:
brinks_pad[2]

array([   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,   

In [167]:
test_pad[1]

array([    0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     2,  1619,   685,  1619,     3,   249,
          14,   146,    65,  3498,  1308,   153,   227,   610,   184,
           1,    15,

In [0]:
def b_make_predictions(lstm_size, multiple_fc, fc_units, checkpoint):
    '''Predict the sentiment of the testing data'''
    
    # Record all of the predictions
    all_preds = []

    model = build_rnn(n_words = n_words, 
                      embed_size = embed_size,
                      batch_size = batch_size,
                      lstm_size = lstm_size,
                      num_layers = num_layers,
                      dropout = dropout,
                      learning_rate = learning_rate,
                      multiple_fc = multiple_fc,
                      fc_units = fc_units) 
    
    with tf.Session() as sess:
        saver = tf.train.Saver()
        # Load the model
        saver.restore(sess, checkpoint)
        test_state = sess.run(model.initial_state)
        for _, x in enumerate(get_test_batches(brinks_pad, batch_size), 1):
            feed = {model.inputs: x,
                    model.keep_prob: 1,
                    model.initial_state: test_state}
            predictions = sess.run(model.predictions, feed_dict=feed)
            for pred in predictions:
                all_preds.append(float(pred))
                
    return all_preds

In [0]:
checkpoint1 = "gdrive/My Drive/Model_checkpoints/trial2/sentiment_ru=128,fcl=False,fcu=256.ckpt"
checkpoint2 = "gdrive/My Drive/Model_checkpoints/trial2/sentiment_ru=128,fcl=False,fcu=128.ckpt"
checkpoint3 = "gdrive/My Drive/Model_checkpoints/trial2/sentiment_ru=64,fcl=True,fcu=256.ckpt"

In [170]:
# Make predictions using the best 3 models
brinks_predictions1 = b_make_predictions(128, False, 256, checkpoint1)
brinks_predictions2 = b_make_predictions(128, False, 128, checkpoint2)
brinks_predictions3 = b_make_predictions(64, True, 256, checkpoint3)

INFO:tensorflow:Restoring parameters from gdrive/My Drive/Model_checkpoints/trial2/sentiment_ru=128,fcl=False,fcu=256.ckpt
INFO:tensorflow:Restoring parameters from gdrive/My Drive/Model_checkpoints/trial2/sentiment_ru=128,fcl=False,fcu=128.ckpt
INFO:tensorflow:Restoring parameters from gdrive/My Drive/Model_checkpoints/trial2/sentiment_ru=64,fcl=True,fcu=256.ckpt


In [171]:
len(brinks_predictions1)

1250

In [0]:
brinks_predictions_combined1 = (pd.DataFrame(brinks_predictions1) + pd.DataFrame(brinks_predictions2) + pd.DataFrame(brinks_predictions3))/3

In [178]:
brinks_predictions_combined1

,0
0,0.528725
1,0.416811
2,0.405012
3,0.950734
4,0.374442
5,0.964146
6,0.401709
7,0.353279
8,0.492759
9,0.552810


In [179]:
brinks_test.head(10)

,Review ID,Review
0,1,
1,2,Talk to customers about updating equipment.
2,3,When calling in to cancel an account you sho...
3,4,A pair monthly fee and my security system doe...
4,5,"battery defect in central unit, did call and ..."
5,6,CHEAPER MONTHLY RATES!!!
6,7,I have not Brinks service since approx June 2...
7,8,I would like someone to come give me a new si...
8,9,Some contacts for my representative and produ...
9,10,Up to date sine


In [0]:
brinks_final = brinks_test

In [0]:
brinks_final['predictions'] = brinks_predictions_combined1

In [185]:
brinks_final.head()

,Review ID,Review,predictions
0,1,,0.528725
1,2,Talk to customers about updating equipment.,0.416811
2,3,When calling in to cancel an account you sho...,0.405012
3,4,A pair monthly fee and my security system doe...,0.950734
4,5,"battery defect in central unit, did call and ...",0.374442


In [0]:
brinks_final.to_excel("brinks_output.xlsx")

In [0]:
from google.colab import files
files.download('brinks_output.xlsx')